In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from pycine.raw import read_frames
from scipy import signal
import skimage
# from skimage.measure import label, regionprops, regionprops_table
from skimage.filters import threshold_otsu, sobel, try_all_threshold, threshold_local, threshold_triangle, threshold_minimum
# from skimage.morphology import closing, square
# from skimage.color import label2rgb
# from skimage.segmentation import clear_border
# from skimage.exposure import equalize_hist, equalize_adapthist
import cv2 as cv
import re
import glob

##### The following functions are used to find cine files on drive, load them, and then process them to a state that allows ptaps to be found.

In [11]:
def find_files(pattern):
    """Use regex to locate desired files
    Returns list of file paths.
    """
    cine_files = glob.glob(r"D:\Raw_DPSP-Data/*/*.cine")
    pattern = re.compile(pattern)
    matches = [file for file in cine_files if pattern.search(file) is not None]
    
    return matches

def load_cine(file, start_frame=0, n_frames=1):
    """Transforms cine file into np.array."""
    cine_file = file
    start_frame = start_frame
    count = n_frames
    raw_images, setup, bpp = read_frames(cine_file, start_frame=start_frame, count=count)
    images = np.array([next(raw_images) for _ in range(n_frames)],dtype=np.float32)
    
    return images


def array_norm(image):

    im = image.copy()
    im_max = im.max()
    im_min = im.min()
    im_norm = (im-im_min)/(im_max-im_min)

    return im_norm


# functions used for automating ptaps assignment and creating an array surrounding each one to determine lumo 

def thresh_image(im_norm, thresh=0.05):
    """sets all pixels lower than threshold to 1 and everything else to zero.
    """

    im_norm = im_norm.copy()
    im_norm_max = im_norm.max()
    im_norm[im_norm > thresh] = im_norm_max*2 # arb number used as intermediate stage
    im_norm[im_norm <= thresh] = 1
    im_norm[im_norm == im_norm_max*2] = 0
    
    return im_norm


def process_image(img_raw):
    """get image ready for cv2 circle finder."""

    img = array_norm(img_raw)

    # create multiple images at different thresholds then compile into one image
    im_thresh_lst = []
    for i in np.arange(0.005,0.1,0.005):
        im_thresh = thresh_image(img, thresh=i)
        im_thresh_lst.append(im_thresh)

    labs = np.sum(im_thresh_lst, axis=0)

    # blur out noise
    img = skimage.filters.gaussian(labs, sigma=2, )
    img = array_norm(img)

    # cv takes 8 bit image
    img = img*100 
    img = img.astype(np.uint8) 
    
    return img

In [50]:
ptap_df = pd.read_csv("ptap_locations_and_distances_all_configs.csv", index_col=0) #### important to define index column

def get_ptap_df(path):
    if re.search("Empty-Bay", file) is not None:
        file_ptap_df = ptap_df.loc[ptap_df['configuration']=='no_store']
    elif re.search("Store-in", file) is not None:
        file_ptap_df = ptap_df.loc[ptap_df['configuration']=='store_in']
    elif re.search("Store-out", file) is not None:
        file_ptap_df = ptap_df.loc[ptap_df['configuration']=='store_out']
    
#     display(file_ptap_df.head())
    return file_ptap_df

In [51]:
def find_distance(coords):
    """Calculate distance between each set of coordinates in a list.
    input: list of n tuples of shape (2,)
    output: matrix with shape (n,n)."""
    
    if not isinstance(coords, np.ndarray):
        coords = np.array(coords)
    
    x, y = coords.T
    x = x[:, np.newaxis]
    y = y[:, np.newaxis]
    distance = np.sqrt((x-x.T)**2 + (y-y.T)**2)
    return distance


def find_circles(img_processed, max_dist_between_circs=50, param1=25,param2=15, minRadius=0,maxRadius=10):
    global cimg
    circles_all = cv.HoughCircles(img_processed,cv.HOUGH_GRADIENT,1,max_dist_between_circs,
                                param1=param1,param2=param2, minRadius=minRadius,maxRadius=maxRadius)
    if circles_all is not None:
        circles_all = np.uint16(np.around(circles_all[0]))
        # superimpose cimg with circles found
        for i in circles_all:
            # draw the outer circle
            cv.circle(cimg,(i[0],i[1]),i[2],(0,255,0),2)
            # draw the center of the circle
            cv.circle(cimg,(i[0],i[1]),2,(255,0,0),3)
    circle_df = pd.DataFrame(circles_all, columns=['x', 'y', 'r'])
    circle_df = circle_df.sort_values('x', ascending=False).reset_index(drop=True)

    return circle_df


def find_circle_distances(circle_df):

    circle_df = circle_df
    circle_coords = [*zip(circle_df['x'], circle_df['y'])]

    # find distance between circles
    distance_mat = find_distance(circle_coords)

    dist_df = pd.DataFrame(np.array(distance_mat).reshape(len(circle_df),len(circle_df)))

    # df gives coords of ptap and distance from other ptaps
    df_find = circle_df.join(dist_df)

    return df_find

def score_circles(file, circle_distance_df, circle_df, verbose=False):
    """ Scoring unlabelled circle distances against ptap distances to determine if circle is a ptap"""
    match_df = pd.DataFrame()
    
    ptap_df = get_ptap_df(file)

    only_ptap_columns = ~ptap_df.columns.isin(['x', 'y', 'r', 'ptap', 'configuration'])
    all_ptap_dists = ptap_df.loc[:,only_ptap_columns].dropna(axis=1, ) # remove ptaps that aren't found config

    # iterate through set of distances for each unlabelled circle
    for i in range(len(circle_distance_df)):
        unlabelled_dists = np.around(circle_distance_df.iloc[i, 4:].values.astype('float32'),1)
        if verbose:
            print(f"\nresults for index {i} tap\n==========================")

        # iterate through set of distances for each ptap
        for j in range(len(ptap_df)):  
            ptap_name = ptap_df.iloc[j].ptap
            ptap_dists = np.around(all_ptap_dists.iloc[j,:].values.astype('float32'), 1) 

            # create a bool matrix where unknown circle dist sets are compared to ptap dist sets.
            mask = np.isclose(unlabelled_dists.reshape(-1,1), ptap_dists, atol=5) # true if unlabelled distance matches ptap dist
            ptap_dists_broadcast = np.broadcast_to(ptap_dists, (len(unlabelled_dists),len(ptap_dists))) # need to broadcast to apply mask
            matches = set(ptap_dists_broadcast[mask]) # returns matching distances. duplicates removed through set 
            match_rating = round(100*len(matches)/len(ptap_dists),0)
            match_df.loc[i, ptap_name] = match_rating
            
            if verbose:
                print(f"ptap {ptap_name} has a {match_rating}% match")


    highest_scoring_ptap = match_df.apply(lambda row: row.idxmax(), axis=1)
    highest_score = match_df.apply(lambda row: row.values.max(), axis=1)
    match_df['highest_scoring_ptap'] = highest_scoring_ptap
    match_df['highest_score'] = highest_score


    # A table showing how closely linked each circle is to a ptap
    match_df = match_df.join(circle_df).loc[match_df.highest_score>70] # remove any circles with a score less than 70
    match_df['config'] = file
    match_df['config'] = match_df.config.str.strip(r'D:\Raw_DPSP-Data\\')
    
    return match_df

def get_ptap_luminosity_means(file, matched_circles_df, frame_count=100):
    """100 frames gives good results. no need to use more."""
    img_gen, _, _ = read_frames(file, start_frame=0, count=frame_count) # around 6600 frames for r'D:\Raw_DPSP-Data\62040-DPSP-Empty-Bay-doors-off-no-LE-TE-Beta0\MBH009_62040-M000_Beta_0_CineF10.cine'

    circles_filtered = matched_circles_df.loc[:,['x', 'y', 'r', 'highest_scoring_ptap']].values

    luminosity_df = pd.DataFrame()
    for i in range(frame_count):
        img_raw = np.array(next(img_gen), dtype=np.float32)
        cimg2 = cv.cvtColor(array_norm(img_raw),cv.COLOR_GRAY2BGR) 
        cimg2*=2.5
        # Iterate through each ptap -> create surrounding ring -> record luminosity at each ring pixel -> take mean -> add to df 
        for j, circle in enumerate(circles_filtered):
            cv.circle(cimg2,(circle[0],circle[1]),circle[2]+7,(0,255,0),4) # ring has inner radius of 7-4/2 and outer of 7+4/2
            ring_coords = np.argwhere(cimg2[:,:,1]==255)
            ring_luminosity = img_raw[ring_coords[:,0], ring_coords[:,1]]
            luminosity_df.loc[i, circle[-1]] = ring_luminosity.mean() # mean of each pixel

        del img_raw


    luminosity_mean = luminosity_df.apply(lambda x: x.mean()) # mean of ptap across frames
    luminosity_mean_df = pd.pivot_table(luminosity_mean.reset_index(), columns='index')
    luminosity_mean_df['config'] = file
    
    return luminosity_df,luminosity_mean_df

In [52]:
def show_ptap_assignment(img_raw, img, circle_coords, matched_circles_df):
    """Plots images showing processing carried out to identify ptaps."""
    cimg2 = cv.cvtColor(img,cv.COLOR_GRAY2BGR)
    circles_filtered = matched_circles.loc[:,['x', 'y', 'r']].values

    for i in circles_filtered:
        cv.circle(cimg2,(i[0],i[1]),i[2],(0,255,0),2)
            # draw the center of the circle
        cv.circle(cimg2,(i[0],i[1]),2,(255,0,0),3)
    
    kwargs = {'aspect':'equal'}
    
    fig, axs = plt.subplots(1,4, figsize=(20,10))
    axs[0].imshow(img_raw, **kwargs)
    axs[1].imshow(img, **kwargs)
    axs[2].imshow(cimg, **kwargs)
    axs[3].imshow(cimg2, **kwargs)

    for i in range(4):    
        axs[i].invert_xaxis()

    # annotate unlabelled circle plot
    for i, coord in enumerate(circle_coords):
        axs[2].text(*coord, circle_df.index[i], c='r', size=20)   


    # annotate labelled filtered circle plot
    ptap_coords = [*zip(matched_circles.x, matched_circles.y)]
    for i, coord in enumerate(ptap_coords):
        axs[3].text(*coord, matched_circles.highest_scoring_ptap.values[i], c='r', size=20)

    plt.show()

### Putting functions together
The following cell uses all of the necessary functions to locate ptaps and determine mean luminosity for a given configuration.


In [63]:
pat = "M" # All files
files = find_files(pat)

ptap_mean_luminosity_df = pd.DataFrame()
for file in files:
    print(file)
    ims = load_cine(file)
    img_raw = ims[0,:,:]
    img = process_image(img_raw)
    cimg = cv.cvtColor(img,cv.COLOR_GRAY2BGR)

    circle_df = find_circles(img, param2=10)
    circle_coords = [*zip(circle_df['x'], circle_df['y'])]
    circle_distance_df = find_circle_distances(circle_df)
    matched_circles = score_circles(file, circle_distance_df, circle_df, verbose=False)
    _,ptap_mean_lum_df = get_ptap_luminosity_means(file, matched_circles)
    
    ptap_mean_luminosity_df = pd.concat((ptap_mean_luminosity_df, ptap_mean_lum_df))


#     show_ptap_assignment(img_raw, img, circle_coords, matched_circles)





D:\Raw_DPSP-Data\62040-DPSP-Empty-Bay-doors-off-no-LE-TE-Beta0\MBH009-62040_M_05-B_0.cine
D:\Raw_DPSP-Data\62040-DPSP-Empty-Bay-doors-off-no-LE-TE-Beta0\MBH009_62040-M000_Beta_0_CineF10.cine
D:\Raw_DPSP-Data\62040-DPSP-Empty-Bay-doors-off-no-LE-TE-Beta0\MBH009_62040-M050_Beta_0_CineF3.cine
D:\Raw_DPSP-Data\62040-DPSP-Empty-Bay-doors-off-no-LE-TE-Beta0\MBH009_62040-M070_Beta_0_CineF4.cine
D:\Raw_DPSP-Data\62040-DPSP-Empty-Bay-doors-off-no-LE-TE-Beta0\MBH009_62040-M080_Beta_0_CineF5.cine
D:\Raw_DPSP-Data\62040-DPSP-Empty-Bay-doors-off-no-LE-TE-Beta0\MBH009_62040-M080_Beta_0_CineF6.cine
D:\Raw_DPSP-Data\62040-DPSP-Empty-Bay-doors-off-no-LE-TE-Beta0\MBH009_62040-M085_Beta_0_CineF7.cine
D:\Raw_DPSP-Data\62040-DPSP-Empty-Bay-doors-off-no-LE-TE-Beta0\MBH009_62040-M090_Beta_0_CineF8.cine
D:\Raw_DPSP-Data\62040-DPSP-Empty-Bay-doors-off-no-LE-TE-Beta0\MBH009_62040-M095_Beta_0_CineF9.cine
D:\Raw_DPSP-Data\62040-DPSP-Empty-Bay-doors-off-no-LE-TE-Beta0\MBH009_Cam_17109_CineF1.cine
D:\Raw_DPSP-Data\

In [68]:
luminosity_df = ptap_mean_luminosity_df.reset_index(drop=True)
luminosity_df.to_csv('mean_luminosity.csv')

# Configuration Image Findings
Comments based on first frame only.
- For store out and focus on store, camera is further from model than for other configurations.
- Different luminosity for store-in and store-out configs (store-in brighter). is this due to setup or degradation of paint?


### Determining the impact variation in exposure has on average luminosity
After testing ten random configurations, the effects of longer exposure (i.e. using more frames) has negligible effect on the average the mean luminosity value for any of the ptaps.
Testing was carried out by taking the mean of the ptap means at a given frame interval, and measuring how the affect of more frames altered this result.
means were taken at [100, 250, 500, 1000, 2000, 3000, 4000, 5000] and mean_value_at_x_interval/mean_value_max varied by less than 0.5% for all tested configurations.  

As a result, I am confident that using the mean of 100 frames is representative of the mean for all frames.   

For wind-off configurations, ten frames is a more than sufficient number for giving the representative mean.

In [ ]:
def validate_frame_selection():
    """Method used to determine how many frames to use for mean values.
    Takes a random sample of ten configurations to test.
    ***pickle file exists containing results***
    """
    pat = "M[-_]?0(?!00)"
    files = find_files(pat)
    permuatated_file_list = np.random.permutation(files)
    df_lst = []
    for file in permuatated_file_list[:10]:
        print(file)
        ims = load_cine(file)
        img_raw = ims[0,:,:]
        img = process_image(img_raw)

        circle_df = find_circles(img)
        circle_coords = [*zip(circle_df['x'], circle_df['y'])]
        circle_distance_df = find_circle_distances(circle_df)
        matched_circles = score_circles(circle_distance_df, circle_df, verbose=False)
        matched_circles
        ptap_luminosity_5000frames,_ = get_ptap_luminosity_means(file, matched_circles, 5000)
        df_lst.append(ptap_luminosity_5000frames)
        empty_df = pd.DataFrame()
        for i in [100, 250, 500, 1000, 2000, 3000, 4000, 5000]:
            luminosity_mean = ptap_luminosity_5000frames[:i].apply(lambda x: x.mean()) # mean of ptap across frames
            luminosity_mean_df = pd.pivot_table(luminosity_mean.reset_index(), columns='index')
            luminosity_mean_df[i]=i
            luminosity_mean_df.set_index(i, inplace=True)
            empty_df = pd.concat((empty_df, luminosity_mean_df))

        empty_df['mean_norm'] = empty_df.apply(lambda row: row.mean(), axis=1)
        empty_df['mean_norm'] = empty_df.mean_norm / empty_df['mean_norm'].max()
        display(empty_df)
        empty_df['mean_norm'].plot()
        plt.show()